In [58]:
%load_ext Cython
%time
%cython

ERROR:root:Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
CPU times: user 22 µs, sys: 14 µs, total: 36 µs
Wall time: 1.3 ms


In [59]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [60]:
import sqlite3
conn = sqlite3.connect('data/capstone.db')
# tblGame , tblMovie, tblReview, tblTVShow

In [64]:
# load games dataset
games = pd.read_sql_query("select * from tblGame;", conn)
games['gameID'] = games.index + 1
games.head(5)

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
0,Ginger: Beyond the Crystal,/game/playstation-4/ginger-beyond-the-crystal,http://static.metacritic.com/images/products/g...,Drakhar Studio,Action,E,2016-10-25 00:00:00,"A world destroyed by a crystal explosion, a my...",1
1,JoJo's Bizarre Adventure: Eyes of Heaven,/game/playstation-4/jojos-bizarre-adventure-ey...,http://static.metacritic.com/images/products/g...,CyberConnect2,Action,T,2016-06-28 00:00:00,"Featuring a new two-on-two battle system, larg...",2
2,The Crew,/game/playstation-4/the-crew,http://static.metacritic.com/images/products/g...,"Ubisoft Reflections, Ivory Tower",Racing,T,2014-12-02 00:00:00,Strap in for a ride that will find you infiltr...,3
3,Batman: Arkham Knight - A Matter of Family,/game/playstation-4/batman-arkham-knight---a-m...,http://static.metacritic.com/images/products/g...,"Rocksteady Studios, WB Games Montreal",Action Adventure,M,2015-07-14 00:00:00,When The Joker kidnaps Commissioner Gordon and...,4
4,8DAYS,/game/playstation-4/8days,http://static.metacritic.com/images/products/g...,Santa Clara Games,Action,M,2017-02-07 00:00:00,The world has been dominated by dark forces si...,5


In [65]:
# Read in Critic Reviews
reviews = pd.read_sql_query("select * from tblReview;", conn)


In [66]:
# subset for just critic reviews to build a model
subset = reviews[reviews['reviewType']=='c']
subset = df[['gameID','text']]
subset = subset.dropna()
subset.head(5)

,gameID,text
0,1.0,Ginger: Beyond the Crystal is a very good plat...
1,1.0,"It\s not a bad game at all, but it has technic..."
2,1.0,Ginger: Beyond the Crystal tries to revisit th...
3,1.0,It may not offer much of a challenge or much i...
4,1.0,"It may sound like I’m being harsh on Ginger, b..."


In [67]:
# get a unique review corpus
review_docs = subset.groupby(['gameID'])['text'].apply(lambda x: ''.join(x)).reset_index()
review_docs.head(5)

,gameID,text
0,1.0,Ginger: Beyond the Crystal is a very good plat...
1,2.0,JoJo’s Bizarre Adventure: Eyes in Heaven featu...
2,3.0,The Crew does a lot of things right. Not only ...
3,4.0,Playing as Batgirl is not quite as interesting...
4,5.0,The frantic nature of the gameplay coupled wit...


In [68]:
stopwrds = stopwords.words('english')

In [69]:
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

In [71]:
# clean up review docs, add utf8 encoding
review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)

In [72]:
# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(review_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(review_docs.iloc[i,2]), tags=['%s' % i])

In [73]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save("data/review.model")

In [74]:
# testing similar words
print model.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('princess', 0.5597556829452515), ('rerelease', 0.5508875250816345), ('remix', 0.49537473917007446), ('timeless', 0.49250730872154236), ('runic', 0.48749983310699463)]


In [75]:
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a dataframe with top_n products with cosine similarity score
def recommend(l,n):
    # get indexes of top_n similar items 
    index = top_n_index(l, n)
    
    # return list of products 
    top_products = copy.deepcopy(review_docs.iloc[index,][['gameID']])
    top_products['index']= top_products.index
    
    # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(index)):
        # note - this results in a bug: top_products['cossim'].loc[index[i],] = sim_array[index[i]]
        top_products.loc[index[i],'cossim'] = sim_array[index[i]]
        
    return top_products   

In [89]:
games.loc[games['name'].str.contains('Overwatch')]

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
14589,Overwatch,/game/pc/overwatch,http://static.metacritic.com/images/products/g...,Blizzard Entertainment,General,T,2016-05-23 00:00:00,Overwatch is a highly stylized team-based shoo...,14590
17748,Overwatch,/game/xbox-one/overwatch,http://static.metacritic.com/images/products/g...,Blizzard Entertainment,Action,T,2016-05-23 00:00:00,Overwatch is a highly stylized team-based shoo...,17749
17865,Overwatch,/game/playstation-4/overwatch,http://static.metacritic.com/images/products/g...,Blizzard Entertainment,Action,T,2016-05-23 00:00:00,Overwatch is a highly stylized team-based shoo...,17866


In [93]:
# "Basic recommendation system with doc2vec

# Select gameID of a product you like
gameID = 17866
input_vec = model.docvecs[review_docs.loc[review_docs['gameID']==gameID].index[0]]

# Calculated a cossim matrix between input product and model's vectors
sim_array = map(lambda v: cossim(input_vec, v), model.docvecs)

#recommendation
print "You liked this product::: %s" %games.loc[games['gameID']==gameID]['name']
res = recommend(sim_array,7)
res['gameID'] = res.apply(lambda row: int(row['gameID']),axis = 1)
pd.merge(res, games[['gameID','name']], on='gameID')

You liked this product::: 17865    Overwatch
Name: name, dtype: object


,gameID,index,cossim,name
0,15286,15259,0.716596,Section 8: Prejudice
1,17749,17717,0.718418,Overwatch
2,19446,19414,0.722208,Plants vs Zombies: Garden Warfare
3,5717,5714,0.735982,Battlefield 1943
4,18013,17981,0.740918,Titanfall
5,12690,12666,0.751953,Splatoon
6,14590,14564,0.782060,Overwatch


In [94]:
#save a pickle file
import pickle

with open('models/model_critic_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [95]:
# to load a pickle file
#with open('models/model_critic_only.pickle', 'rb') as f:
#    entry = pickle.load(f)   